In [11]:
import numpy as np
import os
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
from helper import  read_data
from helper import  read_label
from helper import isgpu

device = isgpu()


GPU is available


In [12]:
cd D:\Program\machineLearning\Appendicitis\

D:\Program\machineLearning\Appendicitis


In [13]:
from  models.Unet import Basic
model = Basic(in_channels=1).to(device)

In [14]:
import json
appendix = input("請輸入要獲取的編號:")
modelname = model.__class__.__name__
filename = f"params/{modelname}/{modelname}{appendix}"

if not os.path.isfile(filename+'.pth'):
    print(f"{filename}.pth not exist.")
else:
    print(model.load_state_dict(torch.load(filename+'.pth')))
with open(f'{filename}.json', 'r') as f:
    params = json.load(f)
        

<All keys matched successfully>


In [15]:
cd D:\Program\data\AOCR2024\

D:\Program\data\AOCR2024


In [16]:
data = read_data("TrainValid_Image/train_data", shift=30, termi=10)
df = read_label("TrainValid_ground_truth.csv")
df.set_index('id', inplace=True)

processed = []
for key, value in data.items():
  scan  = df.loc[df.index.str.startswith(key+'_')]
  labels = np.array(scan['label'])
  processed.append((value, labels))
  
# 轉換每個掃描中的圖像和標籤
images_list = []
labels_list = []

for value, label in processed:
    # value.shape 為 (512, 512, n)，label.shape 為 (n,)
    for i in range(value.shape[2]):
        image = value[:, :, i]
        image_tensor = torch.from_numpy(image).float().unsqueeze(0)  # 添加通道維度
        label_tensor = torch.tensor(label[i], dtype=torch.float32)
        images_list.append(image_tensor)
        labels_list.append(label_tensor)

# 合併成批次數據
images = torch.stack(images_list)
labels = torch.stack(labels_list)

In [17]:
# 評估設置
num_epochs = params['num_epochs']
batch_size = params['batch_size']
num_batches = len(images) // batch_size

In [18]:
model.eval()  
with torch.no_grad():  # 不更新梯度
    correct = 0
    total = 0
    for i in range(num_batches):
        batch_images = images[i*batch_size:(i+1)*batch_size].to(device)
        batch_labels = labels[i*batch_size:(i+1)*batch_size].to(device)

        outputs = model(batch_images)
        predicted = (outputs.squeeze() > 0.5).float()  

        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy on the data: {accuracy * 100:.2f}%')

Accuracy on the data: 96.01%
